# Host Code Optimization Techniques

In this part of the lab we are going to look at two ways of optimizing the host code to reduce the amount of time the hardware spends idle:

 1. Overlapping compute and communication
 2. Using the hardware scheduler to pipeline operations
 
We're going to look at how we can profile and time the host code from within Jupyter and what effect these optimizations have.

For this notebook we are going to keep on using the same bitstream consisting of vector add (_vadd_) and vector multiply (_vmult_) hardware kernels.

In [ ]:
import pynq

ol = pynq.Overlay('intro.xclbin')

vadd = ol.vadd_1
vmult = ol.vmult_1

## Overlapping compute and communication

There are three stages for both vector operations in this design; a data copy to the board, the computation, and a data copy of the results back to the host. The goal of this optimization is to  perform data-copying while the hardware kernel is running, rather than running all these stages sequentially. If we consider calling `vadd` once there is no overlap we can exploit as there are direct dependencies between the copying and the kernel execution.

![Single iteration of vadd](img/vadd_single.svg)

Opportunity to apply this optimization generally arises when a kernel is called as some part of a looped or iterated process. Once we add a second iteration as follows:

![Two iterations of vadd with no overlap](img/vadd_2_no_overlap.svg)

we can see the empty space in the timing diagram where we can optimize.

![Two iterations of vadd with overlap](img/vadd_2_overlap.svg)

Note that each iteration still takes the same amount of time from start to finish but we can pack the iterations closer together to reduce the total execution time. 

To test this optimization we are going to use an array of 8 million elements that we can split into 8 parts. Using the numpy-style array syntax we can create the same buffers as previously but with an extra dimension of 8.

In [ ]:
in_a = pynq.allocate((8,1024,1024), 'u4')
in_b = pynq.allocate((8,1024,1024), 'u4')
out_c = pynq.allocate((8,1024,1024), 'u4')

To get some baselines lets create two functions: one which carries out the vector addition on the entire array and another function that calls the kernel 8 times but doesn't attempt any overlap.

In [ ]:
def vadd_all_at_once(a, b, c):
    a.sync_to_device()
    b.sync_to_device()
    vadd.call(a, b, c, 8*1024*1024)
    c.sync_from_device()
    
def vadd_no_overlap(a, b, c):
    for i in range(8):
        a[i].sync_to_device()
        b[i].sync_to_device()
        vadd.call(a[i], b[i], c[i], 1024*1024)
        c[i].sync_from_device()

The `%timeit` magic that is built into Jupyter can be used to estimate the time taken for each function. Using `%timeit` will run a single line multiple times to try and establish an accurate mean and standard deviation for the execution time of the time. There is also a `%%timeit` variant which operates on entire cells.

In [ ]:
%timeit vadd_all_at_once(in_a, in_b, out_c)
%timeit vadd_no_overlap(in_a, in_b, out_c)

We can see that the version with the loop is marginally slower than the all-at-once version which is to be expected as they are performing the exact same task but the looped version has additional overhead.

Now we can think about how to overlap the memory transfers (i.e. the sync functions) with the kernel execution. Looking again at the execution diagram we can focus on the middle cell to see what our loop needs to look like.

![Vector addition with highlight](img/vadd_highlight.svg)

From the diagram we can see that the order of operations in our main loop is going to involve:

 1. Start the hardware kernel
 2. Copy out the data for the previous iteration
 3. Copy in input data for the next iteration
 4. Wait for the kernel to complete

We'll also need some logic to handle the first and last iterations

### Asynchronous execution

So far we have been using the `.call` function to run the kernel and wait for completion. To perform the overlapping we need to start the kernel and have the ability to execute other code before waiting for the kernel to complete. In PYNQ this is done using the `.start` function which has the same arguments as `.call` but immediately returns a _wait handle_. A wait handle is an object with a `.wait()` function which blocks until the execution is complete.

Putting all this together we can write the implementation of vector addition which fully overlaps the compute and communication.

In [ ]:
def vadd_overlap(a, b, c):
    # Transfer the first input blocks
    a[0].sync_to_device()
    b[0].sync_to_device()
    
    for i in range(8):
        wh = vadd.start(a[i], b[i], c[i], 1024*1024)
        if i != 0:
            c[i-1].sync_from_device()
        if i != 7:
            a[i+1].sync_to_device()
            b[i+1].sync_to_device()
        wh.wait()
        
    # Transfer the last output block
    c[7].sync_from_device()

Again we can use `%timeit` to see how our new overlapped vadd compares to the previous iterations

In [ ]:
%timeit vadd_overlap(in_a, in_b, out_c)

From the results, we can see that there is a small improvement. This technique works best when the time needed to execute the hardware kernel and the time to transfer its data are approximately equal. It is less effective as the difference between the data transfer time and compute time increases. In the next notebook we will have a look at a compression example where the data transfer time and computer time are more balanced than vector addition. In the vector addition, the computation is very simple and the time to transfer data dominates.

## Pipelining Accelerators

The second optimization is pipelining the execution of multiple hardware blocks. As an example consider the vector multiply accumulate (VMAC) from the previous notebook. If we were to call it in a loop the resulting hardware execution would look something like:

![Vector MAC with no overlap](img/vmac_no_overlap.svg)

which only uses each hardware kernel 50% of the time. Ideally we'd like an execution schedule that looks more like:

![Vector MAC with overlap](img/vmac_overlap.svg)

where multiple iterations of the loop can be using different kernels at the same time.

The first consideration is how much temporary space we're going to need. As two loop iterations will be operating simultaneously we need to make sure they each have an independent buffer to work on. We can allocate a 2 million element temporary buffer we will ping-pong between.

In [ ]:
temp = pynq.allocate((2,1024,1024), 'u4')

We'll also need a result buffer

In [ ]:
acc = pynq.allocate((1024,1024), 'u4')

Now we can write our multiple-accumulate loop. Again to create a base-line we'll start with a simple looped version with no overlapping. We're going to ignore the data-transfer times for now to get a direct view on how efficiently the hardware kernels are being used.

In [ ]:
def vmac_plain(a, b, acc):
    for i in range(8):
        vmult.call(a[i], b[i], temp[0], 1024*1024)
        vadd.call(temp[0], acc, acc, 1024*1024)

As a first pass at optimization we can rewrite the loop in the same way as for the overlapped communication example

In [ ]:
def vmac_overlapped(a, b, acc):
    # Perform the first multiplication
    vmult.call(a[0], b[0], temp[0], 1024*1024)
    # Loop over all of the additions
    for i in range(8):
        wh = vadd.start(acc, temp[i%2], acc, 1024*1024)
        if i != 7:
            vmult.call(a[i+1], b[i+1], temp[(i+1)%2], 1024*1024)
        wh.wait()

We can test the two versions again using `%timeit`

In [ ]:
%timeit vmac_plain(in_a, in_b, acc)
%timeit vmac_overlapped(in_a, in_b, acc)

This approach doesn't really scale as we increase the number of stages. We also need software running on the host to hand the computation off from the `vmult` to `vadd` kernels. The time it takes for the vmult kernel to signal completion to the host and have the host start the vadd kernel introduces additional latency. For many platforms, including Alveo and F1, there is a low-level scheduler we can use to help both simplify our loop and reduce the time between tasks being started.

### Using `waitfor`

Both the `.call` and `.start` functions have an optional keyword argument of `waitfor` which takes a list of wait handles which must complete before the kernel can start. `.start` will still return immediately. On some platforms the scheduling of the accelerators is performed using a processor in the FPGA resulting if much faster task turn-around than having to round-trip through software before the next accelerator can be started.

Starting lots of tasks does potentially lead to some non-determinism in the order in which tasks are executed - any ready task can be scheduled when an accelerator becomes idle so we need to enforce some dependencies to make sure that no two tasks try and use the same buffer. Going back to our diagram

![Vector MAC with overlap](img/vmac_overlap.svg)

The green and blue iterations are both going to use the first temporary buffer so we must wait until the blue iteration's vadd has finished before starting the green iteration. We can do this by using a queue of wait handles.

In [ ]:
import collections

def vmac_waitfor(a, b, acc):
    handles = collections.deque()
    for i in range(8):
        # Get the handle of two iterations ago
        prev_handle = handles.popleft() if len(handles) >= 2 else None
        mult_handle = vmult.start(a[i], b[i], temp[i%2], 1024*1024,
                                  waitfor=(prev_handle,))
        add_handle = vadd.start(acc, temp[i%2], acc, 1024*1024,
                                waitfor=(mult_handle,))
        handles.append(add_handle)
    # Wait for the last handle to complete
    handles[-1].wait()

This approach has the buffer as the focus as the loop body compared with the previous approach which put the execution of one of the accelerators as the focus of the loop. Focusing on the buffers can be more intuitive for long pipelines of functions at the cost of making it harder to reason about what's going on at a particular point in time.

Running the function we can see there is very little difference between the two VMAC implementations.

In [ ]:
%timeit vmac_waitfor(in_a, in_b, acc)

## Exercises

So far we have looked at these two optimizations in isolation and we haven't been verifying the output of our functions.

 1. Check that the vmac implementations actually accumulate correctly
 2. Create a version of vmac that uses both pipelined execution and overlapped communication
 3. (Optional) Explore and graph the effect of the buffer size on the merits of each approach.
 
Again, once you're finished shut down the kernel using the _Kernel_ menu to make sure the device is free for the other notebooks.

Copyright (C) 2020 Xilinx, Inc